In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


PREVIOUS STEP IS USED TO MOUNT THE CONTENTS OF GOOGLE DRIVE

In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


**SET THE TRAINING AND TESTING PATH**

In [0]:
train_path = '/content/gdrive/My Drive/Colab Notebooks/Images/train' 
valid_path = '/content/gdrive/My Drive/Colab Notebooks/Images/valid'
test_path = '/content/gdrive/My Drive/Colab Notebooks/Images/test'

In [0]:
train_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=25)
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=25)
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=25, shuffle=False)

Found 1073 images belonging to 6 classes.
Found 138 images belonging to 6 classes.
Found 133 images belonging to 6 classes.


**MAKING THE MODEL**

In [0]:
mobile = keras.applications.mobilenet.MobileNet()

W0621 05:33:18.758264 139989429757824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0621 05:33:18.809906 139989429757824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0621 05:33:18.839854 139989429757824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0621 05:33:18.896111 139989429757824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0621 05:33:18.897786 1399894297

17227776/17225924 [==============================] - 0s 0us/step


In [0]:
x = mobile.layers[-6].output
predictions = Dense(6, activation='softmax')(x)

In [0]:
mobile.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [0]:
model = Model(inputs=mobile.input, outputs=predictions)

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [0]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

W0621 05:36:09.076109 139989429757824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



**LOADING WEIGHTS FROM MY TRAINED MODEL WHICH HAS THE ORIGINAL MOBILENET ARCHITECHTURE EXCEPT THE LAST 6 LAYERS AND IS MADE BY TRAINING THE LAST 20 LAYERS OF THE MODEL.
AFTER EXPERIMENTATION, THE BEST ACCURACY IS OBTAINED AFTER 80 EPOCHS WITH A LEARNING RATE OF 0.0001 USING THE ADAM OPTIMIZER**

In [0]:
model.load_weights('/content/gdrive/My Drive/Colab Notebooks/file_new5_model.h5')

**PREDICTING AGAINST THE TEST IMAGES WHICH GIVES 90.3% ACCURACY**

In [0]:
test_labels = test_batches.classes
predictions = model.predict_generator(test_batches, steps=6, verbose=2)
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))
print(cm)

[[ 7  1  1  0  2  0]
 [ 1  3  1  1  1  1]
 [ 0  0  6  0  0  0]
 [ 0  0  0  1  2  0]
 [ 0  0  0  0 95  0]
 [ 0  0  0  0  2  8]]
